## Imports

In [230]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.actions.action_builder import ActionBuilder
import pyautogui as pag
import time
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
import json
import pandas as pd

## Request

In [231]:
os.environ['PATH'] += r"D:/Anaconda/anaconda3"
driver = webdriver.Firefox()
driver.maximize_window()

In [232]:
driver.get('https://permits.cityofboise.org/CitizenAccess/Welcome.aspx')


In [233]:
link = driver.find_element(By.XPATH, '/html/body/form/div[3]/header/div/nav/div/div/ul/li[2]/span/div')
link.click()

WebDriverWait(driver,30).until(
    EC.text_to_be_present_in_element(
        (By.XPATH,'/html/body/form/div[3]/header/div/nav/div/div/ul/li[2]/div/a[5]'),
        'Licenses'
    )
)

lk2 = driver.find_element(By.XPATH, '//*[@id="cobnav"]/div/ul/li[2]/div/a[2]') 

lk2.click()



In [234]:
rec_type = driver.find_element(By.ID, 'ctl00_PlaceHolderMain_generalSearchForm_ddlGSPermitType')



rec_type.click()

sel_rec = driver.find_element(By.XPATH, '/html/body/form/div[4]/div/div[7]/div[1]/table/tbody/tr/td/div[2]/div[3]/div/div/div[2]/div[1]/div/div[1]/div[1]/div[1]/div[1]/div[2]/table/tbody/tr/td/div/div[2]/table/tbody/tr/td[1]/select/option[2]')

sel_rec.click()


## Calendar

In [235]:
scroll_origin = ScrollOrigin.from_viewport(10, 10)
ActionChains(driver)\
.scroll_from_origin(scroll_origin, 0, 800)\
.perform()

In [236]:
time.sleep(4)

In [237]:
# Values changes for each monitor
pag.moveTo(480,250)
pag.click()
time.sleep(1)
pag.moveTo(440,280)
pag.click()
time.sleep(1)
pag.moveTo(390,350)
pag.click()
time.sleep(1)
pag.moveTo(430,340)
pag.click()
time.sleep(1)

In [238]:
find_eng = driver.find_element(By.ID, 'ctl00_PlaceHolderMain_btnNewSearch')
driver.execute_script("document.getElementById('ctl00_PlaceHolderMain_btnNewSearch').click()")

## Search

In [239]:
scroll_origin = ScrollOrigin.from_viewport(0, 100)
ActionChains(driver)\
.scroll_from_origin(scroll_origin, 0, -10**10)\
.perform()

time.sleep(2)
pag.scroll(-100000)

## Permits scraping

In [240]:
all_permits = {'0':{}}

In [241]:
## Status

def p_status (value):
    global status
    status = None

    try:
        status = driver.find_element(By.ID, f"ctl00_PlaceHolderMain_dgvPermitList_gdvPermitList_ctl{value}_lblStatus").text
        
        
    except:
        pass

## Project Name

def p_name (value):
    global project_name
    project_name = None

    try:
        project_name = driver.find_element(By.ID, f"ctl00_PlaceHolderMain_dgvPermitList_gdvPermitList_ctl{value}_lblProjectName").text

    except:
        pass

## Permit Number
def p_num (value):
    global permit_num
    permit_num = None

    if value is not None:
        try:
            permit_num = driver.find_element(By.ID, f"ctl00_PlaceHolderMain_dgvPermitList_gdvPermitList_ctl{value}_lblPermitNumber1").text
            print(permit_num, '| Features: High')
            return True
            
        except:
           permit_num = driver.find_element(By.ID, f"ctl00_PlaceHolderMain_dgvPermitList_gdvPermitList_ctl{value}_lblPermitNumber").text
           print(permit_num, '| Features: Low')
           return False
        

def go_to_link(value):

    if p_num(value) == True:
        try:
            driver.execute_script(f"document.getElementById('ctl00_PlaceHolderMain_dgvPermitList_gdvPermitList_ctl{value}_hlPermitNumber1').click()")
            return True
        except:
            driver.execute_script(f"document.getElementById('ctl00_PlaceHolderMain_dgvPermitList_gdvPermitList_ctl{value}_hlPermitNumber').click()")
            return True
    else:
        return False



## Project attributes
     
def attributes(value):


    global project_st
    global project_city
    global licenced_professional
    global professional_city
    global professional_zip
    global professional_street
    global applicant_name
    global applicant_company
    global applicant_main_phone
    global applicant_email   
    
    
    project_st = None
    project_city = None
    licenced_professional = None
    professional_city = None
    professional_zip = None
    professional_street =None
    applicant_name = None
    applicant_company = None
    applicant_main_phone = None
    applicant_email = None


    if go_to_link(value) == True:
        time.sleep(7)
        try:
            project_st = driver.find_element(By.CLASS_NAME, f"contactinfo_addressline1").text

        except:
            project_st = driver.find_element(By.CLASS_NAME, f"contactinfo_addressline2").text


        
        
        ## Project City
        try:
            project_city = driver.find_element(By.CLASS_NAME, f"contactinfo_region").text

        except:
            pass

        

        ## Applicant Name
        try:
            apl_fname = driver.find_element(By.CLASS_NAME, f"contactinfo_firstname").text
            apl_lname = driver.find_element(By.CLASS_NAME, f"contactinfo_lastname").text

            applicant_name = apl_fname + ' ' + apl_lname
            

        except:
            pass
            
        
        ## Applicant Company Name
        try:
            applicant_company = driver.find_element(By.CLASS_NAME, f"contactinfo_businessname").text

        except:
            pass


        ## Applicant Phone
        try:
            applicant_main_phone = driver.find_element(By.CLASS_NAME, f"contactinfo_phone1").text
            
            if type(applicant_main_phone) == str:
                applicant_main_phone = driver.find_element(By.CLASS_NAME, f"contactinfo_phone1").text
                applicant_main_phone = applicant_main_phone.split(sep="\n")
                applicant_main_phone = applicant_main_phone[1]
            else:
                applicant_main_phone = driver.find_element(By.CLASS_NAME, f"contactinfo_phone1").text

        except:
            pass
            


        ## Applicant Email
        try:
            applicant_email = driver.find_element(By.CLASS_NAME, f"contactinfo_email").text

        except:
            pass


        ## Licensed Professional
        try:
            table = driver.find_elements(By.CLASS_NAME, f"table_child")[2].text
        except:
            pass
        
        
        splitted = table.split(sep="\n")

        if len(splitted) >= 3:
            try:
                if ':' in splitted[3].split(sep=',')[0].strip():
                    professional_city=(splitted[2].split(sep=',')[0])
                    professional_street=(splitted[1])
                    licenced_professional=(splitted[0])    
                else:
                    professional_city=(splitted[3].split(sep=',')[0])
                    professional_street=(splitted[2])
                    licenced_professional=(splitted[1])                

            except:
                professional_zip=(splitted)
   
        driver.back()




def permits():
    # for i in range(len(values)):
        global permits_data

        permits_data = {}
        permits_data['permit_num']= permit_num
        permits_data['project_name']= project_name
        permits_data['status']= status
        permits_data['project_st']= project_st
        permits_data['project_city']= project_city
        permits_data['licenced_professional']= licenced_professional
        permits_data['professional_city']= professional_city
        permits_data['professional_zip']= professional_zip
        permits_data['professional_street']= professional_street
        permits_data['applicant_name']= applicant_name
        permits_data['applicant_company']= applicant_company
        permits_data['applicant_main_phone']= applicant_main_phone
        permits_data['applicant_email']= applicant_email
        add_permits(permits_data)


def add_permits(permits_data):
    all_permits.update({f'{i}': (permits_data)})


In [242]:
pag.moveTo(30,1040)
pag.click()
scroll_origin = ScrollOrigin.from_viewport(0, 100)
ActionChains(driver)\
.scroll_from_origin(scroll_origin, 0, -10**10)\
.perform()

time.sleep(2)
pag.scroll(-100000)

In [243]:
values = ['02','03','04','05','06']#,'07', '08','09', '10','11', '12','13']
for i, value in enumerate(values):
    print('Working on {}...'.format(value))
    p_status(value)
    p_name(value)
    attributes(value)
    permits()
    add_permits(permits_data)


Working on 02...
24TMP06331 | Features: Low
Working on 03...
BLD24-00429 | Features: High
Working on 04...
BLD24-00420 | Features: High
Working on 05...
BLD24-00418 | Features: High
Working on 06...
BLD24-00415 | Features: High


## Output

In [244]:
print(all_permits)


{'0': {'permit_num': '24TMP06331', 'project_name': '', 'status': None, 'project_st': None, 'project_city': None, 'licenced_professional': None, 'professional_city': None, 'professional_zip': None, 'professional_street': None, 'applicant_name': None, 'applicant_company': None, 'applicant_main_phone': None, 'applicant_email': None}, '1': {'permit_num': 'BLD24-00429', 'project_name': '4388 W Hill Rd', 'status': 'Prescreen', 'project_st': 'po box 6152', 'project_city': 'Boise,', 'licenced_professional': 'BARROW DESIGN AND CONSTRUCTION LLC', 'professional_city': 'BOISE', 'professional_zip': None, 'professional_street': '1317 W RANCH RD', 'applicant_name': 'robert barrow', 'applicant_company': None, 'applicant_main_phone': '208-572-6076', 'applicant_email': 'barrowdc@ymail.com'}, '2': {'permit_num': 'BLD24-00420', 'project_name': 'SR4_2113', 'status': 'Prescreen', 'project_st': '2490 E Gala St', 'project_city': 'Meridian,', 'licenced_professional': 'BOISE, ID, United States', 'professional_c

In [245]:
r = json.dumps(all_permits)
load = json.loads(r)


In [246]:
df = pd.json_normalize(load['0'])

len(load.keys())

5

In [247]:
if len(load.keys())>1:

    for i in range (0,len(load.keys())-1):
        a = (pd.json_normalize(load[f'{i+1}']))
        b = [df, a]
        output = pd.concat (b)
        df = output


else:
    df = pd.json_normalize(load['0'])
    output = df

display(output)

## Export
# output.to_csv

,permit_num,project_name,status,project_st,project_city,licenced_professional,professional_city,professional_zip,professional_street,applicant_name,applicant_company,applicant_main_phone,applicant_email
0,24TMP06331,,None,None,None,None,None,None,None,None,None,None,None
0,BLD24-00429,4388 W Hill Rd,Prescreen,po box 6152,"Boise,",BARROW DESIGN AND CONSTRUCTION LLC,BOISE,None,1317 W RANCH RD,robert barrow,None,208-572-6076,barrowdc@ymail.com
0,BLD24-00420,SR4_2113,Prescreen,2490 E Gala St,"Meridian,","BOISE, ID, United States",208-914-0518,None,Main Phone:,Josh Leighton,Eaglewood Homes,208-854-7353,josh@eaglewood.com
0,BLD24-00418,Harris Ranch North #4 LOT 1 BLOCK 4,Applicant Upload,729 S. Bridgeway Pl.,"Eagle,",None,None,"[Jeff Harris, BOISE, ID, United States, Respon...",None,Kyle Davis,Boise Hunter Homes,208-577-5501,bhhdrafting@boisehunterhomes.com
0,BLD24-00415,Wolke Residence,Prescreen,1456 E IRON EAGLE DR,"EAGLE,",SHERBURNE-MARRS LLC,EAGLE,None,1456 E IRON EAGLE DR,Desiree Hammond,SHERBURNE-MARRS LLC,208-939-9221,desiree@sherburne-marrs.com
